In [ ]:
import pandas as pd
import numpy as np
import scipy.signal
import os
import glob

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
#Global variables
sampling_rate=512;
bin_size=10;
#Time window to show
t1=2;t2=5;

In [ ]:
df=pd.read_json("../all_unfiltered_mean_AE_Swiss-Short.json",orient="records")
df.head()

In [ ]:
# no_of_timepoints=len(df.loc[0]['mean_AE'])
# time=np.arange(0,no_of_timepoints)/(60*sampling_rate)

# fig= make_subplots(rows=8, cols=2,vertical_spacing=0.01,horizontal_spacing=0.01,shared_xaxes='all',shared_yaxes='all',subplot_titles=("ID1","ID2","ID3","ID4","ID5","ID6","ID7","ID8","ID9","ID10","ID11","ID12","ID13","ID14","ID15","ID16"))

# pid=1
# for ridx in range(1,9):
#     for cidx in range(1,3):
#         tim=time[(t1*60*sampling_rate):(t2*60*sampling_rate)]
#         mean=np.array(df.loc[pid-1]['mean_AE'][(t1*60*sampling_rate):(t2*60*sampling_rate)])
#         #stdd=np.array(df.loc[pid-1]['std_aa'][(2*60*sampling_rate):(5*60*sampling_rate)])


#         fig.add_trace(go.Scatter(x=tim,y=mean,
#                                 mode='lines',
#                                 line=dict(width=0.5),
#                                 showlegend=False,
#                                 ),row=ridx,col=cidx)
#         pid+=1


# fig.update_xaxes(title_text="Time (in Mins)",range=(t1,t2),row=all,col=all)
# fig.update_yaxes(title_text="average AE",range=(2,5.5),row=all,col=all)
# fig.add_shape(dict(type="line",x0=3,y0=2,x1=3,y1=5.5,line=dict(color="red", width=3)),row="all",col="all")

# fig.add_shape(dict(type="line",x0=2.5,y0=2,x1=2.5,y1=5.5,line=dict(color="green", dash='dashdot',width=1)),row="all",col="all")
# fig.add_shape(dict(type="line",x0=4,y0=2,x1=4,y1=5.5,line=dict(color="green",dash='dashdot',width=1)),row="all",col="all")

# fig.update_layout(
#         template="simple_white",
#         legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5,font=dict(size=40)),
#         font_family="Times new Roman",font_color="black",font_size=25,height=1800,width=1500)


# fig.show()
# fig.write_image("../images/Fig4C.png")
# fig.write_html("interactive_Figures/Fig4C.html")

In [ ]:
no_of_timepoints=len(df['mean_AE'][0])
time=np.arange(0,no_of_timepoints)/(60*sampling_rate)

#replacing None with np.nan
df['mean_AE'] = df['mean_AE'].apply(lambda x: [np.nan if val is None else val for val in x])
df['std_AE'] = df['std_AE'].apply(lambda x: [np.nan if val is None else val for val in x])

In [ ]:
mean_AE= np.nanmean(np.array(df['mean_AE'].tolist()),axis=0)
std_AE = np.nanstd(np.array(df['std_AE'].tolist()),axis=0,ddof=1)
std_AE = np.where(np.isnan(std_AE),0.0, std_AE) # replacing np.nan with 0.0

time=time[(t1*60*sampling_rate):(t2*60*sampling_rate)]
mean_AE=mean_AE[(t1*60*sampling_rate):(t2*60*sampling_rate)]
std_AE=std_AE[(t1*60*sampling_rate):(t2*60*sampling_rate)]

In [ ]:
# #fig= make_subplots(rows=2, cols=1,vertical_spacing=0.1,shared_yaxes="all")
# fig=go.Figure()



# fig.add_trace(go.Scatter(x=time,y=mean_AE,mode='lines',
#                         line=dict(color='rgb(31, 119, 180)'),
#                         showlegend=False))

# fig.add_trace(go.Scatter(x=time,y=mean_AE+std_AE,
#         mode='lines',marker=dict(color="#444"),line=dict(width=0),showlegend=False))

# fig.add_trace(go.Scatter(x=time,y=mean_AE-std_AE,
#                          marker=dict(color="#444"),line=dict(width=0),mode='lines',
#                          fillcolor='rgba(68, 68, 68, 0.3)',fill='tonexty',showlegend=False))


# fig.update_yaxes(title_text="average AE",range=(2.8,5))
# fig.update_xaxes(title_text="Time (in Mins)",range=(t1,t2))
# fig.add_shape(dict(type="line",x0=3,y0=2.8,x1=3,y1=5,line=dict(color="red", width=3)))

# fig.add_shape(dict(type="line",x0=2.5,y0=2.8,x1=2.5,y1=5,line=dict(color="black", dash='dashdot',width=1)))
# fig.add_shape(dict(type="line",x0=3.5,y0=2.8,x1=3.5,y1=5,line=dict(color="black", dash='dashdot',width=1)))

# fig.update_layout(
#         template="plotly_white",
#         legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5,font=dict(size=40)),
#         font_family="Times new Roman",font_color="blue",font_size=25,height=600,width=1200)


# fig.show()
# fig.write_image("../Link to img/Fig4A.png")
# fig.write_html("interactive_Figures/Fig4A.html")

In [ ]:
st=2.5;ut=4;
outp=pd.read_json("../all_unfiltered_electrode_data_Swiss-Short_"+str(st)+"_"+str(ut)+".json",orient="records")
outp.head()

In [ ]:
output={'pat_ID':[],'elec_no':[],'X_t1':[],'X_t2':[],'elecs_t1':[],'elecs_t2':[],'ampen_t1':[],'ampen_t2':[]}

for pid in range(1,17):
    s=outp.query('pat_id=="ID'+str(pid)+'"')[['ampen_t1','elecs_t1','X_t1','X_t2','ampen_t2','elecs_t2']];
    
    #replacing 0s with np.nan
    s['elecs_t1'] = s['elecs_t1'].apply(lambda x: [np.nan if val==0.0 else val for val in x])
    #replacing 0s with np.nan
    s['elecs_t2'] = s['elecs_t2'].apply(lambda x: [np.nan if val==0.0 else val for val in x])
    #Not bothering with X_t1,X_t2 and ampen_t1,ampen_t2 as they are same and don't have 0s

    output['pat_ID'].append("ID"+str(pid))
    output['elec_no'].append(outp.query('pat_id=="ID'+str(pid)+'"')['elec_no'].unique()[0])
    
    output['X_t1'].append(np.nanmean(np.array(s['X_t1'].tolist()),axis=0))
    output['X_t2'].append(np.nanmean(np.array(s['X_t2'].tolist()),axis=0))
  
    output['elecs_t1'].append(np.nanmean(np.array(s['elecs_t1'].tolist()),axis=0))
    output['elecs_t2'].append(np.nanmean(np.array(s['elecs_t2'].tolist()),axis=0))
    
    output['ampen_t1'].append(s['ampen_t1'].mean())
    output['ampen_t2'].append(s['ampen_t2'].mean())

outS=pd.DataFrame.from_dict(output)
outS.head()

In [ ]:
y1m = np.nanmean(np.array(outS['elecs_t1'].tolist()),axis=0)
x1m = np.nanmean(np.array(outS['X_t1'].tolist()),axis=0)
y2m = np.nanmean(np.array(outS['elecs_t2'].tolist()),axis=0)
x2m = np.nanmean(np.array(outS['X_t2'].tolist()),axis=0)

In [ ]:
fig = make_subplots(
    rows=2, cols=2,shared_yaxes='rows',vertical_spacing=0.1,horizontal_spacing=0.01,
    specs=[[{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=("Time T1","Time T2", ""))

fig.add_trace(go.Bar(name='t1', x=x1m, y=y1m,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(name='t2', x=x2m[:len(x1m)], y=y2m[:len(x1m)],showlegend=False),row=1,col=2)


fig.add_trace(go.Scatter(x=time,y=mean_AE,mode='lines',
                        line=dict(color='blue'),
                        showlegend=False),row=2,col=1)

fig.add_trace(go.Scatter(x=time,y=mean_AE+std_AE,
        mode='lines',marker=dict(color="#444"),line=dict(width=0),showlegend=False),row=2,col=1)

fig.add_trace(go.Scatter(x=time,y=mean_AE-std_AE,
                         marker=dict(color="#444"),line=dict(width=0),mode='lines',
                         fillcolor='rgba(75,   0, 130, 0.3)',fill='tonexty',showlegend=False),row=2,col=1)

fig.update_yaxes(title_text="average AE",range=(2.8,5),row=2,col=1)
fig.update_xaxes(title_text="Time (in Mins)",row=2,col=1)

fig.add_shape(dict(type="line",x0=3,y0=2.8,x1=3,y1=5,line=dict(color="red", width=3)),row=2,col=1)

fig.add_shape(dict(type="line",x0=2.5,y0=2.8,x1=2.5,y1=5,line=dict(color="black", dash='dashdot',width=1)),row=2,col=1)
fig.add_shape(dict(type="line",x0=4,y0=2.8,x1=4,y1=5,line=dict(color="black", dash='dashdot',width=1)),row=2,col=1)

fig.add_annotation(dict(text="mean AE "+str(np.round(outS['ampen_t1'].mean(),decimals=2)),xref='x',yref='y',
                x=800,y=0.14,showarrow=False),
                font=dict(size=30, color="blue",family="Times new Roman"),
                row=1, col=1)
fig.add_annotation(dict(text="mean AE "+str(np.round(outS['ampen_t2'].mean(),decimals=2)),xref='x',yref='y',
                x=800,y=0.14,showarrow=False),
                font=dict(size=30, color="blue",family="Times new Roman"),
                row=1, col=2)

fig.add_annotation(dict(text="T1",xref='x',yref='y',
        x=2.5,y=4.5,showarrow=False),font=dict(size=30, color="blue",family="Times new Roman"),
        row=2, col=1)

fig.add_annotation(dict(text="T2",xref='x',yref='y',
        x=4,y=3.5,showarrow=False),font=dict(size=30, color="blue",family="Times new Roman"),
        row=2, col=1)

fig.add_annotation(dict(text="seizure onset",xref='x',yref='y',
        x=3,y=4.5,showarrow=False),font=dict(size=30, color="blue",family="Times new Roman"),
        row=2, col=1)

fig.update_layout(
        template="plotly_white",
        #legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5,font=dict(size=40)),
        font_family="Times new Roman",font_color="black",font_size=25,height=900,width=1500)

fig.update_traces(textposition='top right',row=2,col=1)

fig.show()
# fig.write_image("../Link to img/Fig4.png")
#fig.write_html("interactive_Figures/Fig4.html")



Indiv Plots

In [ ]:
fig=make_subplots(rows=1, cols=2,horizontal_spacing=0.01,shared_yaxes="all")


# fig.add_trace(go.Bar(name='t1', x=x1m, y=y1m),row=1,col=1)
# fig.add_trace(go.Bar(name='t2', x=x2m[:len(x1m)], y=y2m[:len(x1m)]),row=1,col=2)


fig.add_trace(go.Bar(name='t1', x=x1m, y=y1m),row=1,col=1)
fig.add_trace(go.Bar(name='t2', x=x2m, y=y2m),row=1,col=2)







fig.update_layout(
        template="plotly_white",
        legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5,font=dict(size=40)),
        font_family="Times new Roman",font_color="blue",font_size=25,height=600,width=1200)




In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f,ax=plt.subplots(figsize=(20,10))
ax.bar(x1m[~np.isnan(y1m)],y1m[~np.isnan(y1m)],width=2,color='red',label="t="+str(st)+"min")
ax.bar(x2m[~np.isnan(y2m)]+5,y2m[~np.isnan(y2m)],width=2,color='blue',label="t="+str(ut)+"min")
ax.set_title("ID"+str(pid),fontsize=20)
ax.legend()
ax.grid()
ax.set_xlim([0,2000])
textstr = '\n'.join((
    r'$\mathrm{AE_{t1}}=%.2f$' % (outS['ampen_t1'].mean()),
    r'$\mathrm{AE_{t2}}=%.2f$' % (outS['ampen_t2'].mean())))
# place a text box in upper left in axes coords
ax.text(0.9, 0.90, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top')
    

In [ ]:
for pid in range(1,17):
    s=outp.query('pat_id=="ID'+str(pid)+'"')[['ampen_t1','elecs_t1','X_t1','X_t2','ampen_t2','elecs_t2']]
    #replacing 0s with np.nan
    s['elecs_t1'] = s['elecs_t1'].apply(lambda x: [np.nan if val==0.0 else val for val in x])
    #replacing 0s with np.nan
    s['elecs_t2'] = s['elecs_t2'].apply(lambda x: [np.nan if val==0.0 else val for val in x])
    #Not bothering with X_t1,X_t2 and ampen_t1,ampen_t2 as they are same and don't have 0s

    y1m = np.nanmean(np.array(s['elecs_t1'].tolist()),axis=0)
    x1m = np.nanmean(np.array(s['X_t1'].tolist()),axis=0)
    y2m = np.nanmean(np.array(s['elecs_t2'].tolist()),axis=0)
    x2m = np.nanmean(np.array(s['X_t2'].tolist()),axis=0)
    
    f,ax=plt.subplots(figsize=(20,10))
    ax.bar(x1m[~np.isnan(y1m)],y1m[~np.isnan(y1m)],width=2,color='red',label="t="+str(st)+"min")
    ax.bar(x2m[~np.isnan(y2m)],y2m[~np.isnan(y2m)],width=2,color='blue',label="t="+str(ut)+"min")
    ax.set_title("ID"+str(pid),fontsize=20)
    ax.legend()
    ax.grid()
    textstr = '\n'.join((
        r'$\mathrm{AE_{t1}}=%.2f$' % (s['ampen_t1'].mean()),
        r'$\mathrm{AE_{t2}}=%.2f$' % (s['ampen_t2'].mean())))
    # place a text box in upper left in axes coords
    ax.text(0.9, 0.90, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top')
    
    f.tight_layout()
    f.savefig("../images/Fig4Bs/ID"+str(pid)+".png")
    f.savefig("Isa_See_this/ID"+str(pid)+".png")
    plt.close(f)